In [48]:
import pandas as pd

# read in

In [49]:
df_small = pd.read_csv('../../2_Feature_Engineering/export/randomized_balanced_dataset.csv')
df_small.head()

,raw_text,severity,binary_label
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False


In [50]:
df_big = pd.read_csv('../../2_Feature_Engineering/export/combined_iwg_fb-tw_polly.csv')
df_big.head()

,raw_text,severity,binary_label
0,bitte nicht die #Türkei zum #EU-Mitglied mache...,0.0,False
1,Wieso bekommen #rapefugees mehr als unsere Har...,0.0,False
2,#Asylanteninvasion Wenn es auf unseren Straßen...,0.0,False
3,745 Millionen Menschen leben in #Europa. Ca. 4...,0.0,False
4,Tja die #SPD will unsere Steuergelder für Flüc...,0.0,False


# clean

In [51]:
import re
# remove links and user hashes
def clean_tweet(text):
    text = re.sub(r'http\S+', '', text) # links
    text = re.sub(r'^[a-f0-9]{16}', '', text) # user hashes
    text = re.sub(r'@(\w+)',  '', text) # usernames
    text = re.sub(r'RT',  '',text) # RT --> retweets
    return text

In [52]:
df_small['cleaned_text'] = df_small['raw_text'].apply(clean_tweet)
df_big['cleaned_text'] = df_big['raw_text'].apply(clean_tweet)

In [53]:
from model_helpers import clean_input as ci

In [54]:
df_small['cleaned_text'] = df_small['cleaned_text'].apply(ci)
df_big['cleaned_text'] = df_big['cleaned_text'].apply(ci)

# vectorize

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
tfidf_vec = TfidfVectorizer(ngram_range=(2,5), analyzer='char', stop_words='german', max_features=10000)

In [57]:
small_features = tfidf_vec_small.fit_transform(df_small['cleaned_text'])
big_features = tfidf_vec_big.fit_transform(df_big['cleaned_text'])

# split

In [58]:
from sklearn.model_selection import train_test_split

In [60]:
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(small_features, df_small['binary_label'], test_size=0.2)
X_train_big, X_test_big, y_train_big, y_test_big = train_test_split(big_features, df_big['binary_label'], test_size=0.2)

# train

In [34]:
from sklearn.linear_model import LogisticRegression
lr_small = LogisticRegression() 
lr_big = LogisticRegression()

In [62]:
lr_small.fit(X_train_small, y_train_small)
lr_big.fit(X_train_big, y_train_big)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

# score small dataset

In [64]:
y_pred_small = lr_small.predict(X_test_small)

In [65]:
from sklearn.metrics import precision_recall_fscore_support as score

In [75]:
precision, recall, fscore, support = score(y_test_small, y_pred_small, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben

In [78]:
print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                         round(recall, 3), 
                                                         round((y_pred_small==y_test_small).sum() / len(y_pred_small)),
                                                        round(fscore, 3)))

Precision: 0.809 / Recall: 0.627 / Accuracy: 1.0 / Fscore: 0.707


# score big dataset

In [72]:
y_pred_big = lr_big.predict(X_test_big)

In [79]:
precision, recall, fscore, support = score(y_test_big, y_pred_big, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben

In [81]:
print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                         round(recall, 3), 
                                                         round((y_pred_big==y_test_big).sum() / len(y_pred_big)),
                                                         round(fscore, 3)))

Precision: 0.729 / Recall: 0.72 / Accuracy: 1.0 / Fscore: 0.724


In [85]:
dir(lr_big)

['C',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_estimator_type',
 '_get_param_names',
 '_predict_proba_lr',
 'class_weight',
 'classes_',
 'coef_',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'intercept_scaling',
 'max_iter',
 'multi_class',
 'n_iter_',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_params',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'warm_start']

# export small dataset model

In [82]:
import pickle

In [83]:
pickle.dump(lr_small, open('exports/lr_model.pkl', 'wb'))